#1.Setup for Running Flask App in a Colab

In [ ]:
#Setup for Running Flask App in a Colab
# Install pyngrok and flask
!pip install flask pyngrok tensorflow pillow pandas

# Set your ngrok authtoken (run this once per session)
!ngrok authtoken 2xRZPc8KhsrfRGT2R84fbFuVqn9_6MdCDB2W7b8e31P5mocUh


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


#2.Mount the drive to load the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#3.Temporarily host the project in nygrok

In [ ]:
#real
# Install dependencies (run once)
# !pip install flask pyngrok tensorflow pillow pandas

import os
import io
import numpy as np
import pandas as pd
from flask import Flask, request, render_template_string, redirect, url_for, flash, send_file, session
from tensorflow.keras.models import load_model
from PIL import Image
from pyngrok import ngrok

app = Flask(__name__)
app.secret_key = "supersecretkey"  # Needed for sessions

MODEL_PATH = '/content/drive/MyDrive/InceptionV3.h5'
model = load_model(MODEL_PATH)

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

#Preprocess Image for Model

def preprocess_image(img):
    img = img.resize((128, 128))
    img_array = np.array(img)
    if img_array.shape[-1] == 4:
        img_array = img_array[..., :3]
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Predict Function
def predict(img):
    img_array = preprocess_image(img)
    pred = model.predict(img_array)[0][0]
    label = "Infected" if pred >= 0.5 else "Uninfected"
    return pred, label

#html pages

LANDING_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Malaria Disease Detection</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <script src="https://unpkg.com/@lottiefiles/lottie-player@latest/dist/lottie-player.js"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');
        body {
            font-family: 'Poppins', sans-serif;
            background-image: url("data:image/svg+xml,%3Csvg width='60' height='60' viewBox='0 0 60 60' xmlns='http://www.w3.org/2000/svg'%3E%3Cg fill='none' fill-rule='evenodd'%3E%3Cg fill='%239C92AC' fill-opacity='0.08'%3E%3Cpath d='M36 34v-4h-2v4h-4v2h4v4h2v-4h4v-2h-4zm0-30V0h-2v4h-4v2h4v4h2V6h4V4h-4zM6 34v-4H4v4H0v2h4v4h2v-4h4v-2H6zM6 4V0H4v4H0v2h4v4h2V6h4V4H6z'/%3E%3C/g%3E%3C/g%3E%3C/svg%3E");
        }
    </style>
</head>
<body class="min-h-screen bg-gray-50">
    <!-- Hero Section -->
    <div class="relative overflow-hidden">
        <div class="absolute inset-0 bg-gradient-to-br from-blue-50 to-indigo-100 opacity-90"></div>
        <div class="container mx-auto px-4 py-16 relative">
            <div class="max-w-5xl mx-auto bg-white/80 backdrop-blur-lg rounded-3xl shadow-2xl p-8">
                <div class="flex items-center justify-center mb-8">
                    <lottie-player src="https://assets2.lottiefiles.com/packages/lf20_5njp3vgg.json"
                                 background="transparent" speed="1"
                                 style="width: 120px; height: 120px;" loop autoplay>
                    </lottie-player>
                    <h1 class="text-5xl font-bold text-gray-800 ml-4">
                        Malaria Detection
                    </h1>
                </div>

                <!-- Process Flow -->
                <div class="flex justify-center space-x-6 mb-12">
                    <div class="w-1/3 transform hover:scale-105 transition-all duration-300">
                        <div class="bg-white rounded-xl shadow-lg p-6">
                            <div class="text-center mb-4">
                                <i class="fas fa-upload text-4xl text-blue-600"></i>
                            </div>
                            <h3 class="text-xl font-semibold text-center mb-2">Upload</h3>
                            <p class="text-gray-600 text-sm text-center">Upload blood cell images for analysis</p>
                        </div>
                    </div>
                    <div class="w-1/3 transform hover:scale-105 transition-all duration-300">
                        <div class="bg-white rounded-xl shadow-lg p-6">
                            <div class="text-center mb-4">
                                <i class="fas fa-microscope text-4xl text-indigo-600"></i>
                            </div>
                            <h3 class="text-xl font-semibold text-center mb-2">Analyze</h3>
                            <p class="text-gray-600 text-sm text-center">AI-powered cell analysis</p>
                        </div>
                    </div>
                    <div class="w-1/3 transform hover:scale-105 transition-all duration-300">
                        <div class="bg-white rounded-xl shadow-lg p-6">
                            <div class="text-center mb-4">
                                <i class="fa-solid fa-square-poll-vertical text-4xl text-green-600"></i>
                            </div>
                            <h3 class="text-xl font-semibold text-center mb-2">Results</h3>
                            <p class="text-gray-600 text-sm text-center">Get instant diagnosis results</p>
                        </div>
                    </div>
                </div>

                <!-- Info Cards -->
                <div class="grid grid-cols-2 gap-6 mb-8">
                    <div class="bg-white/80 rounded-xl shadow-lg p-6 transform hover:scale-105 transition-all duration-300">
                        <h3 class="text-xl font-semibold mb-4 flex items-center">
                            <i class="fas fa-robot text-blue-600 mr-2"></i>
                            AI Technology
                        </h3>
                        <p class="text-gray-600">Powered by InceptionV3 deep learning model, our system achieves high accuracy in detecting malaria parasites in blood cells.</p>
                    </div>
                    <div class="bg-white/80 rounded-xl shadow-lg p-6 transform hover:scale-105 transition-all duration-300">
                        <h3 class="text-xl font-semibold mb-4 flex items-center">
                            <i class="fas fa-clock text-indigo-600 mr-2"></i>
                            Real-time Analysis
                        </h3>
                        <p class="text-gray-600">Get instant results with our advanced image processing system that analyzes blood cell images in real-time.</p>
                    </div>
                </div>

                <!-- Action Button -->
                <div class="text-center">
                    <a href="{{ url_for('index') }}"
                       class="inline-flex items-center px-8 py-4 text-lg font-semibold text-white bg-gradient-to-r from-blue-600 to-indigo-600 rounded-full hover:shadow-lg transform hover:scale-105 transition-all duration-300">
                        Start Detection
                        <i class="fas fa-arrow-right ml-2"></i>
                    </a>
                </div>
            </div>
        </div>
    </div>

    <!-- Statistics Section -->
    <div class="container mx-auto px-4 py-12">
        <div class="grid grid-cols-3 gap-8 max-w-4xl mx-auto">
            <div class="bg-white/80 backdrop-blur-lg rounded-xl shadow-lg p-6 text-center transform hover:scale-105 transition-all duration-300">
                <div class="text-4xl font-bold text-blue-600 mb-2">99%</div>
                <div class="text-gray-600">Accuracy Rate</div>
            </div>
            <div class="bg-white/80 backdrop-blur-lg rounded-xl shadow-lg p-6 text-center transform hover:scale-105 transition-all duration-300">
                <div class="text-4xl font-bold text-indigo-600 mb-2">< 2s</div>
                <div class="text-gray-600">Processing Time</div>
            </div>
            <div class="bg-white/80 backdrop-blur-lg rounded-xl shadow-lg p-6 text-center transform hover:scale-105 transition-all duration-300">
                <div class="text-4xl font-bold text-green-600 mb-2">24/7</div>
                <div class="text-gray-600">Availability</div>
            </div>
        </div>
    </div>
</body>
</html>
"""


INDEX_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>Malaria Cell Detection</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" />
</head>
<body class="bg-light">
<div class="container py-5">
    <h1 class="mb-4 text-center">Malaria Blood Smear Cell Detection</h1>

    {% with messages = get_flashed_messages() %}
      {% if messages %}
        <div class="alert alert-danger" role="alert">
          <ul class="mb-0">
          {% for message in messages %}
            <li>{{ message }}</li>
          {% endfor %}
          </ul>
        </div>
      {% endif %}
    {% endwith %}

    <form method="POST" enctype="multipart/form-data" class="border p-4 bg-white rounded shadow-sm">
        <div class="mb-3">
            <label class="form-label me-3">Choose upload type:</label>
            <div class="form-check form-check-inline">
                <input class="form-check-input" type="radio" name="upload_type" id="singleRadio" value="single" checked>
                <label class="form-check-label" for="singleRadio">Single Image</label>
            </div>
            <div class="form-check form-check-inline">
                <input class="form-check-input" type="radio" name="upload_type" id="multipleRadio" value="multiple">
                <label class="form-check-label" for="multipleRadio">Multiple Images</label>
            </div>
        </div>

        <div class="mb-3" id="single_upload">
            <label for="single_file" class="form-label">Upload Single Image</label>
            <input class="form-control" type="file" id="single_file" name="single_file" accept=".jpg,.jpeg,.png" />
        </div>

        <div class="mb-3 d-none" id="multiple_upload">
            <label for="multiple_files" class="form-label">Upload Multiple Images</label>
            <input class="form-control" type="file" id="multiple_files" name="multiple_files" accept=".jpg,.jpeg,.png" multiple />
        </div>

        <button type="submit" class="btn btn-primary w-100">Predict</button>
    </form>
</div>

<script>
    const singleRadio = document.getElementById('singleRadio');
    const multipleRadio = document.getElementById('multipleRadio');
    const singleUpload = document.getElementById('single_upload');
    const multipleUpload = document.getElementById('multiple_upload');

    function toggleUploadFields() {
        if(singleRadio.checked) {
            singleUpload.classList.remove('d-none');
            multipleUpload.classList.add('d-none');
        } else {
            singleUpload.classList.add('d-none');
            multipleUpload.classList.remove('d-none');
        }
    }

    singleRadio.addEventListener('change', toggleUploadFields);
    multipleRadio.addEventListener('change', toggleUploadFields);
</script>

</body>
</html>
"""

RESULT_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>Prediction Results</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" />
</head>
<body class="bg-light">
<div class="container py-5">
    <h1 class="mb-4 text-center">Prediction Results</h1>

    <div class="table-responsive shadow-sm rounded bg-white p-3">
        <table class="table table-bordered table-hover mb-0">
            <thead class="table-primary">
                <tr>
                    <th>Filename</th>
                    <th>Prediction Score</th>
                    <th>Label</th>
                </tr>
            </thead>
            <tbody>
                {% for pred in predictions %}
                <tr>
                    <td>{{ pred.filename }}</td>
                    <td>{{ "%.4f"|format(pred.score) }}</td>
                    <td>{{ pred.label }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>

    <div class="mt-4 text-center">
        {% if not single %}
        <a href="{{ url_for('download_csv') }}" class="btn btn-success me-3">Download CSV</a>
        {% endif %}
        <a href="{{ url_for('index') }}" class="btn btn-secondary">Back to Upload</a>
    </div>
</div>
</body>
</html>
"""
#Route / — landing page
@app.route('/')
def landing():
    return render_template_string(LANDING_HTML)

#Route / — Upload & Predict
@app.route('/index', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        upload_type = request.form.get('upload_type')

        if upload_type == 'single':
            if 'single_file' not in request.files:
                flash('No file part')
                return redirect(request.url)
            file = request.files['single_file']
            if file.filename == '':
                flash('No selected file')
                return redirect(request.url)
            if file and allowed_file(file.filename):
                try:
                    img = Image.open(file.stream)
                    pred_score, pred_label = predict(img)
                    predictions = [{'filename': file.filename, 'score': pred_score, 'label': pred_label}]
                    return render_template_string(RESULT_HTML, predictions=predictions, single=True)
                except Exception as e:
                    flash(f'Error processing image: {e}')
                    return redirect(request.url)
            else:
                flash('Allowed file types: png, jpg, jpeg')
                return redirect(request.url)

        elif upload_type == 'multiple':
            files = request.files.getlist('multiple_files')
            if not files or files[0].filename == '':
                flash('No files selected')
                return redirect(request.url)

            results = []
            for file in files:
                if file and allowed_file(file.filename):
                    try:
                        img = Image.open(file.stream)
                        pred_score, pred_label = predict(img)
                        results.append({'filename': file.filename, 'score': pred_score, 'label': pred_label})
                    except Exception as e:
                        flash(f'Error processing {file.filename}: {e}')
                else:
                    flash(f'Skipped invalid file: {file.filename}')

            if results:
                df = pd.DataFrame(results)
                csv_buffer = io.StringIO()
                df.to_csv(csv_buffer, index=False)
                csv_buffer.seek(0)
                session['csv_data'] = csv_buffer.getvalue()  # Store CSV in session
                return render_template_string(RESULT_HTML, predictions=results, single=False)
            else:
                flash('No valid images uploaded')
                return redirect(request.url)

    return render_template_string(INDEX_HTML)

#Route /download_csv — CSV Download
@app.route('/download_csv')
def download_csv():
    csv_data = session.get('csv_data')
    if not csv_data:
        flash("No CSV data available for download.")
        return redirect(url_for('index'))
    return send_file(io.BytesIO(csv_data.encode()),
                     mimetype='text/csv',
                     as_attachment=True,
                     download_name='malaria_predictions.csv')

# Start ngrok tunnel (make sure you added your authtoken beforehand)
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel URL: {public_url}")

# Run Flask app
app.run(port=5000)


 * ngrok tunnel URL: NgrokTunnel: "https://4138-35-247-64-60.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:47:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:47:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:47:19] "GET /index HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:47:40] "POST /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:48:04] "GET /index HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:49:19] "POST /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:50:11] "GET /index HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:50:25] "POST /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:50:28] "GET /index HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2025 02:51:58] "POST /index HTTP/1.1" 200 -
